In [1]:
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
import fiona
from shapely.geometry import Point, LineString
from rtree import index
import flopy
from GISio import shp2df, df2shp, get_values_at_points, get_proj4
from preproc import NHDdata, renumber_segments, get_upsegs, lines

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
nhd_lines = '../Examples/data/NHDflowlines.shp'
newlines = '../Examples/data/added_lines2.shp'
sfrlines = '../Examples/data/SFRlines2.shp'
grid = '../Examples/data/grid2.shp'
domain = '../Examples/data/domain2.shp'
#dem = 'D:/ATLData/BR/BadRiver/grid/dem/dem_utm_ft/w001001.adf'
dem = '/Users/aleaf/Documents/BR/BadRiver/grid/dem/dem_utm_ft/w001001.adf'

#nhdpathGL = '/Users/aleaf/Documents/NHDPlus/NHDPlusGL/NHDPlus04/NHDPlusAttributes/'
nhdpathGL = '../Examples/data/'

pfvaa_files = nhdpathGL + 'PlusFlowlineVAA.dbf'
pf_files = nhdpathGL + 'PlusFlow.dbf'
elevslope_files = nhdpathGL + 'elevslope.dbf'
nhd_lines_proj4 = get_proj4('/Users/aleaf/Documents/BR/BadRiver/streams/preproc/SFR_lines.shp')

nhd = NHDdata(NHDFlowline=nhd_lines, PlusFlowlineVAA=pfvaa_files, PlusFlow=pf_files,
              elevslope=elevslope_files,
              mf_grid=grid, mf_units='feet',
              model_domain=domain,
              flowlines_proj4=nhd_lines_proj4)

nhd.to_sfr()
nhd.write_linework_shapefile(basename='../Examples/data/SFRlines2')

Reading input...

reading ../Examples/data/elevslope.dbf...
--> building dataframe... (may take a while for large shapefiles)

reading ../Examples/data/PlusFlow.dbf...
--> building dataframe... (may take a while for large shapefiles)

reading ../Examples/data/PlusFlowlineVAA.dbf...
--> building dataframe... (may take a while for large shapefiles)

reading ../Examples/data/NHDflowlines.shp...
--> building dataframe... (may take a while for large shapefiles)

reading ../Examples/data/grid2.shp...
--> building dataframe... (may take a while for large shapefiles)
reprojecting NHDFlowlines from
+proj=longlat +datum=NAD83 +no_defs 
to
+proj=utm +zone=15 +datum=NAD27 +units=m +no_defs ...

clipping flowlines to active area...
intersecting flowlines with grid cells...

Building spatial index...
finished in 1.87s

Intersecting 17 features...
17
finished in 0.10s

setting up segments... (may take a few minutes for large networks)
getting routing information from NHDPlus Plusflow table...
assigni

In [3]:
lns = lines(newlines, mf_grid=grid, mf_grid_node_col='node', model_domain=domain)
lns.get_end_elevs_from_dem(dem)
m1, m2 = lns.append2sfr(nhd.m1, route2reach1=False, routing_tol=200)

Reading input...

reading ../Examples/data/added_lines2.shp...
--> building dataframe... (may take a while for large shapefiles)

reading ../Examples/data/grid2.shp...
--> building dataframe... (may take a while for large shapefiles)
routing new lines within 200 to SFR...

clipping lines to active area...
intersecting lines with grid cells...

Building spatial index...
finished in 1.93s

Intersecting 40 features...
40
finished in 0.01s

setting up reaches and Mat1... (may take a few minutes for large grids)
finished in 0.14s

computing lengths...
computing arbolate sums at reach midpoints...
finished in 0.01s

computing widths...
multiplying length units by 3.28083989501 to convert from GIS to MODFLOW...

setting up Mat2...
finished in 0.00s



In [8]:
df = nhd.m1.append(m1)
df.sort_values(['segment', 'reach'], inplace=True)
df2shp(df, 'temp/junk.shp')

writing temp/junk.shp...


In [4]:
trim_buffer=20
lns = lines(newlines, mf_grid=grid, mf_grid_node_col='node', model_domain=domain)
lns.get_end_elevs_from_dem(dem)
m1, m2 = lns.append2sfr(sfr, route2reach1=False, routing_tol=200)

Reading input...

reading ../Examples/data/added_lines.shp...
--> building dataframe... (may take a while for large shapefiles)

reading ../Examples/data/grid.shp...
--> building dataframe... (may take a while for large shapefiles)
routing new lines within 200 to SFR...

clipping lines to active area...
intersecting lines with grid cells...

Building spatial index...
finished in 6.34s

Intersecting 137 features...
137
finished in 0.06s

setting up reaches and Mat1... (may take a few minutes for large grids)
finished in 1.31s

computing lengths...
computing arbolate sums at reach midpoints...
finished in 0.04s

computing widths...
multiplying length units by 3.28083989501 to convert from GIS to MODFLOW...

setting up Mat2...
finished in 0.00s



In [5]:
nhd = NHDdata(NHDFlowline=nhd_lines, PlusFlowlineVAA=pfvaa_files, PlusFlow=pf_files,
              elevslope=elevslope_files,
              mf_grid=grid, mf_units='feet',
              model_domain=domain,
              flowlines_proj4=nhd_lines_proj4)
              
nhd.to_sfr()

Reading input...

reading /Users/aleaf/Documents/NHDPlus/NHDPlusGL/NHDPlus04/NHDPlusAttributes/elevslope.dbf...
--> building dataframe... (may take a while for large shapefiles)

reading /Users/aleaf/Documents/NHDPlus/NHDPlusGL/NHDPlus04/NHDPlusAttributes/PlusFlow.dbf...
--> building dataframe... (may take a while for large shapefiles)

reading /Users/aleaf/Documents/NHDPlus/NHDPlusGL/NHDPlus04/NHDPlusAttributes/PlusFlowlineVAA.dbf...
--> building dataframe... (may take a while for large shapefiles)

reading ../Examples/data/grid.shp...
--> building dataframe... (may take a while for large shapefiles)
reprojecting NHDFlowlines from
+proj=longlat +datum=NAD83 +no_defs 
to
+proj=utm +zone=15 +datum=NAD27 +units=m +no_defs ...

clipping flowlines to active area...
intersecting flowlines with grid cells...

Building spatial index...
finished in 6.45s

Intersecting 119 features...
119
finished in 0.40s

setting up segments... (may take a few minutes for large networks)
getting routing infor

IndexError: list index out of range

In [6]:
df2shp(m1, 'temp/junk.shp')

writing temp/junk.shp...


In [ ]:
m1 = pd.read_csv('../test/temp/junkMat1.csv')

In [6]:
m1.to_records()

rec.array([ (0, 13912, 1, 29, 1, 0, 0.5192679511000089, 95.23010503448131, 1, 1, 0.037000000000000005, 942),
 (1, 13913, 1, 29, 2, 0, 1.0311761121137468, 259.7254263863121, 1, 1, 0.037000000000000005, 943),
 (2, 13753, 1, 29, 3, 0, 1.4011936841218071, 140.55971321099773, 1, 1, 0.037000000000000005, 944),
 ...,
 (1263, 4688, 1, 47, 13, 0, 3.280839895013123, 262.9583964664946, 1, 1, 0.037000000000000005, 1264),
 (1264, 4688, 1, 47, 14, 0, 3.280839895013123, 140.2498598760475, 1, 1, 0.037000000000000005, 1265),
 (1265, 4848, 1, 47, 15, 0, 3.280839895013123, 123.98980788467978, 1, 1, 0.037000000000000005, 1266)], 
          dtype=[('index', '<i8'), ('node', '<i8'), ('layer', '<i8'), ('segment', '<i8'), ('reach', '<i8'), ('sbtop', '<i8'), ('width', '<f8'), ('length', '<f8'), ('sbthick', '<i8'), ('sbK', '<i8'), ('roughness', '<f8'), ('reachID', '<i8')])

In [35]:
class SFRdata(flopy.modflow.ModflowSfr2):
    
    column_mappings = {'layer': 'k', 
                       'row': 'i', 
                       'column': 'j',
                       'segment': 'iseg', 
                       'reach': 'ireach',
                       'sbtop': 'strtop',
                       }
    def __init__(self, reach_data, segment_data):

[(1, 0.2), (3, 0.5), (5, 0.1), (7, 0.005)]

In [2]:
m1

NameError: name 'm1' is not defined